In [2]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import scipy as sp

In [16]:
def calc_f(y):
    return -y

y0 = -1

def calc_implicit_rk(n):
    g = 1 - 2**0.5 / 2
    c = np.array((g, 1))
    a = np.array((
        (g, 0),
        (1-g, g)
    ))
    b = np.array((1-g, g))

    tmax = 100
    tmin = 0
    h = (tmax - tmin)/n
    t = np.linspace(tmin, tmax, n+1, endpoint=True)
    y = np.zeros((n+1, ))
    # return t.shape, y[0].shape
    y[0] = y0
    k = np.zeros((2,))

    def to_call(yn):
        def to_solve(k):
            ans =  np.array((
                k[0] - calc_f(yn + h * sum(a[0]*k)), # k0y
                k[1] - calc_f(yn + h * sum(a[1]*k))  # k1y
            ))
            # print(ans)
            return ans
        return to_solve

    for i in range(1, n+1):
        to_solve = to_call(y[i-1])
        k = sp.optimize.fsolve(to_solve, (1,1))
        # print(k)
        # y[i] = y[i-1] + h * b @ k
        y[i] = y[i-1] + h*sum(b*k)
    return t, y 


t, y = calc_implicit_rk(2000)
# y = np.clip(y, None, 1e6)
# df = pd.DataFrame({'t': t, 'x': y[:,0], 'z': y[:,1]})
px.line(x=t, y=y)


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.



In [25]:
def calc_fx(x, y, z):
    return -4*x+y

def calc_fy(x, y, z):
    return 2*x-y +z

def calc_fz(x,y,z):
    return -x-z

y0 = np.array((1,2,3))

def calc_implicit_rk(n):
    g = 1 - 2**0.5 / 2
    c = np.array((g, 1))
    a = np.array((
        (g, 0),
        (1-g, g)
    ))
    b = np.array((1-g, g))

    tmax = 100
    tmin = 0
    h = (tmax - tmin)/n
    t = np.linspace(tmin, tmax, n+1, endpoint=True)
    y = np.zeros((n+1, 3))
    y[0] = y0
    k = np.zeros((2,3))

    def to_call(yn):
        def to_solve(flat_k):
            k = flat_k.reshape((2,3))
            ans =  np.array((
                k[0][0] - calc_fx(yn[0] + h * sum(a[0]*k[:,0]), yn[1] + h * sum(a[0]*k[:,1]), yn[2] + h * sum(a[0]*k[:,2])), 
                k[0][1] - calc_fy(yn[0] + h * sum(a[0]*k[:,0]), yn[1] + h * sum(a[0]*k[:,1]), yn[2] + h * sum(a[0]*k[:,2])), 
                k[0][2] - calc_fz(yn[0] + h * sum(a[0]*k[:,0]), yn[1] + h * sum(a[0]*k[:,1]), yn[2] + h * sum(a[0]*k[:,2])), 
                k[1][0] - calc_fx(yn[0] + h * sum(a[1]*k[:,0]), yn[1] + h * sum(a[1]*k[:,1]), yn[2] + h * sum(a[0]*k[:,2])), 
                k[1][1] - calc_fy(yn[0] + h * sum(a[1]*k[:,0]), yn[1] + h * sum(a[1]*k[:,1]), yn[2] + h * sum(a[0]*k[:,2])), 
                k[1][2] - calc_fz(yn[0] + h * sum(a[1]*k[:,0]), yn[1] + h * sum(a[1]*k[:,1]), yn[2] + h * sum(a[0]*k[:,2])), 
            ))
            # print(ans)
            return ans
        return to_solve

    for i in range(1, n+1):
        to_solve = to_call(y[i-1])
        k = sp.optimize.fsolve(to_solve, [1]*6).reshape((2,3))
        y[i][0] = y[i-1][0] + h*sum(b*k[:,0])
        y[i][1] = y[i-1][1] + h*sum(b*k[:,1])
        y[i][2] = y[i-1][2] + h*sum(b*k[:,2])
    return t, y 


t, y = calc_implicit_rk(2000)
# y = np.clip(y, None, 1e6)
# df = pd.DataFrame({'t': t, 'x': y[:,0], 'z': y[:,1]})
px.line(x=t, y=y[:,0]).add_trace(go.Scatter(x=t, y=y[:,1])).add_trace(go.Scatter(x=t, y=y[:,2]))
